<a href="https://colab.research.google.com/github/AiswaryaSrinivas/Hackathon/blob/main/SPacy_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install -U spacy==3.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.0 MB 4.3 MB/s 
     |████████████████████████████████| 660 kB 48.1 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.0
    Uninstalling thinc-8.1.0:
      Successfully uninstalled thinc-8.1.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.0
    Uninstalling spacy-3.4.0:
      Successfully uninstalled spacy-3.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.3.0 requires spacy<3.4.0,>=3.3.0.dev0, but you have spacy 3.2.1 which is incompatible.


In [1]:
!python -m spacy info

/usr/local/lib/python3.7/dist-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)

============================== Info about spaCy ==============================

spaCy version    3.2.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines        en_core_web_sm (3.3.0)        



In [24]:
!wget https://raw.githubusercontent.com/AiswaryaSrinivas/Hackathon/main/annotations.json

--2022-07-15 06:46:18--  https://raw.githubusercontent.com/AiswaryaSrinivas/Hackathon/main/annotations.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1665 (1.6K) [text/plain]
Saving to: ‘annotations.json’

annotations.json    100%[===================>]   1.63K  --.-KB/s    in 0s      

2022-07-15 06:46:18 (22.4 MB/s) - ‘annotations.json’ saved [1665/1665]



In [25]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [26]:
import json
f = open('annotations.json')
TRAIN_DATA = json.load(f)

In [27]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 17/17 [00:00<00:00, 1359.66it/s]


In [28]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --force


⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [29]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy


ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-07-15 06:46:59,170] [INFO] Set up nlp object from config
[2022-07-15 06:46:59,182] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-07-15 06:46:59,186] [INFO] Created vocabulary
[2022-07-15 06:46:59,188] [INFO] Finished initializing nlp object
[2022-07-15 06:46:59,321] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     70.06   20.69   19.15   22.50    0.21
200     200         21.09   1678.30  100.00  100.00  100.00    1.00
400     400          0.00      0.00  100.00  100.00  100.00    1.00
600     600          0.00      0.00  100.00  100

In [30]:
nlp_ner = spacy.load("/content/model-best")

In [35]:
doc=nlp_ner("Nephrology in Bangalore")

In [36]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter


In [51]:
doc=nlp_ner("Dr.Aiswarya  near me Pediatrist")

In [52]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter


In [53]:
doc=nlp_ner("Dr.Aiswarya  Columbia Hospital Pediatrist")

In [54]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter


In [ ]:
/content/model-best

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [57]:
!cp -r "/content/model-best/" "/content/drive/MyDrive/ner_custom/"